# finding missing index

In [ ]:
import pandas as pd
bay=pd.read_csv("../nb_outputs/news_20/baysian_topic_5.csv")
bay.drop(["Unnamed: 0"],axis=1,inplace=True)
bay

In [ ]:
plsa_5top=pd.read_csv("../plsa_outputs/news_20/5_topics.csv")
plsa_5top.drop(["Unnamed: 0"],axis=1,inplace=True)
plsa_5top

# Finding match accuracy

In [ ]:
import pandas as pd

def accuracy(gntm_path,baysian_path,n_largest,match_threshold): #instead of gntm here we have PLSA or other probabilitic models

    bay=pd.read_csv(f"../distribution_finder_news_20/{baysian_path}")
    gnt=pd.read_csv(f"../distribution_finder_news_20/{gntm_path}")
    print("read sucessfully")
    bay.drop(["Unnamed: 0"],axis=1,inplace=True) 
    gnt.drop(["Unnamed: 0"],axis=1,inplace=True) 
    bay,gnt=bay.transpose(),gnt.transpose()
    bay_top_3,gnt_top_3=[],[]

    for col in range(bay.shape[1]):
        try:
            bay_top_3.append(bay.nlargest(n_largest,col).index) #parameter passs
        except:
            pass

    for col in range(gnt.shape[1]):
        try:
            gnt_top_3.append(gnt.nlargest(n_largest,col).index)
        except:
            pass

    bay_col_names=[]
    gnt_col_names=[]
    for i in range(1,n_largest+1):
        bay_col_names.append(f'bay_top{str(i)}')
        gnt_col_names.append(f'gnt_top{str(i)}')
        
    bay_top3_df=pd.DataFrame(bay_top_3,columns=bay_col_names)
    gnt_top3_df=pd.DataFrame(gnt_top_3,columns=gnt_col_names)
    final_matches=[]
    for gnt_out,bay_out in zip(gnt_top_3,bay_top_3):
        matches=[]
        for out in gnt_out:
            if out in bay_out:
                matches.append(True)
            else:
                matches.append(False)
        percentage_of_acc=(matches.count(True)/len(matches))*100
        if percentage_of_acc >= match_threshold: 
            final_matches.append(True)
        else:
            final_matches.append(False)
    result=pd.concat([bay_top3_df,gnt_top3_df,pd.DataFrame(final_matches,columns=["match"])],axis=1)
    # decision=str(input("Do you want to save the o/p(yes/no)?"))
    decision="no"
    if decision=="Yes" or decision=="yes" or decision=="YES":
        file_name=str(input("enter file name :"))
        result.to_csv(f"{file_name}.csv")
    
    try:
        result["match"].value_counts()[True]
    except:
        print('accuracy : 0 % ')
    try:
        result["match"].value_counts()[False]
    except:
        print('accuracy : 100%')

    try:
        accuracy=result["match"].value_counts()[True]/(result["match"].value_counts()[True]+result["match"].value_counts()[False])
        print("The accuracy is :",accuracy)
    except:
        print("result already shown")



topics=str(input("enter no. of topics :"))
n_largest=int(input("enter n_largest : "))
thres=[60,80,100]
for threshold in thres:
# match_threshold=int(input("Enter the threshold value : "))  #80 percentage 
    gntm_path=f"gntm_topic_{topics}.csv"
    baysian_path=f"baysian_topic_{topics}.csv"
    print("topics:",topics)
    print("n largest :",n_largest)
    print("Threshold is :",threshold)
    accuracy(gntm_path,baysian_path,n_largest,threshold)

,0,1,2,3,4
0,0.090909,0.181818,0.181818,0.363636,0.181818
1,0.111111,0.222222,0.222222,0.222222,0.222222
2,0.125000,0.250000,0.125000,0.250000,0.250000
3,0.166667,0.166667,0.333333,0.166667,0.166667
4,0.470588,0.235294,0.029412,0.029412,0.235294
...,...,...,...,...,...
10738,0.071429,0.285714,0.285714,0.071429,0.285714
10739,0.200000,0.200000,0.200000,0.200000,0.200000
10740,0.200000,0.200000,0.200000,0.200000,0.200000
10741,0.363636,0.363636,0.045455,0.045455,0.181818


In [12]:
bay.index
for i,j in 

RangeIndex(start=0, stop=10743, step=1)

In [10]:
plsa_5top=pd.read_csv("../plsa_outputs/news_20/5_topics.csv")
plsa_5top.drop(["Unnamed: 0"],axis=1,inplace=True)
plsa_5top

,index,topic_0,topic_1,topic_2,topic_3,topic_4
0,0,0.182772,0.154299,0.362826,0.265439,0.034664
1,1,0.170013,0.429872,0.280469,0.084446,0.035200
2,2,0.144929,0.126789,0.527657,0.122471,0.078154
3,3,0.063665,0.149297,0.535024,0.229300,0.022714
4,4,0.178789,0.071926,0.572412,0.139527,0.037346
...,...,...,...,...,...,...
18141,18841,0.189119,0.152136,0.315272,0.259054,0.084419
18142,18842,0.160021,0.137618,0.470699,0.194421,0.037241
18143,18843,0.412938,0.144951,0.288788,0.108398,0.044924
18144,18844,0.234518,0.122719,0.284213,0.316082,0.042468
